<a href="https://colab.research.google.com/github/EnaJeong/study-KoELECTRA_fine_tuning/blob/main/KoELECTRA_seq_cls_nsmc_0406.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#드라이브 마운트

Mounted at /content/drive


In [ ]:
import argparse
import json
import logging #로깅 임포트
import os
import glob

import numpy as np
import torch #파이토치 임포트
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
#토치의 DataLoader, RandomSampler, SequentialSampler 임포트
from fastprogress.fastprogress import master_bar, progress_bar

In [ ]:
!pip install attrdict
# ATTtdict > 구현 인스톨, 임포트
from attrdict import AttrDict
from attrdict import AttrDict

In [ ]:
!pip install transformers #트랜스포머도 잊지말고 인스톨

     |████████████████████████████████| 2.0MB 24.1MB/s 
     |████████████████████████████████| 3.3MB 53.0MB/s 
     |████████████████████████████████| 870kB 45.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=cce5822bcb7e0b357e3ded7de9866188fc992e4fcf77cb7a5dd7ee135695ae6f
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
#트랜스포머에서 제공하는 AdamW 사용
#아담 = 운동량+가중치 값
#lr = get_linear_schedule_with_warmup > 러닝레이트 조금씩 감소시키는 스케쥴러


In [ ]:
#소스 끝 

from transformers import ElectraConfig, ElectraTokenizer, ElectraForSequenceClassification

# from src import (
#     CONFIG_CLASSES,
#     TOKENIZER_CLASSES,
#     MODEL_FOR_SEQUENCE_CLASSIFICATION,
#     init_logger,
#     set_seed,
#     compute_metrics
# ) 
#src 에서 데이터를 가지고 오지 않음으로 주석 처리

#자료의 date 설정
def init_logger():
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )

#라벨값과 예상치 평균
def simple_accuracy(labels, preds): #수정
    return (labels == preds).mean()

#라벨값과 예상치 갯수 같을때 리턴 값 발생
def compute_metrics(labels, preds): #수정
    assert len(preds) == len(labels)

    return {
        "acc": simple_accuracy(labels, preds),
    }


#시드 부분도 주자 않으므로 일단 주석 처리
# def set_seed(args):
#     random.seed(args.seed)
#     np.random.seed(args.seed)
#     torch.manual_seed(args.seed)

#     if not args.no_cuda and torch.cuda.is_available():
#         torch.cuda.manual_seed_all(args.seed)

모델 추가

In [ ]:
# 본격적인 NSMC 추가

#Naver sentiment movie corpus = nsmc
#프로세서 클래스 호출
class NsmcProcessor(object):
    """Processor for the NSMC data set """ #데이터 셋의 문장 시작 1

    def __init__(self, args):
        self.args = args

    #라벨값은 0 아님 1
    def get_labels(self):
        return ["0", "1"]

    #엔코딩 utf8 잊지말고 꼭 주기
    @classmethod
    def _read_file(cls, input_file):
        """Reads a tab separated value file."""  #데이터 셋의 문장 시작 2
        with open(input_file, "r", encoding="utf-8") as f:
            lines = [] #빈리스트 주고
            for line in f:
                lines.append(line.strip())
            return lines

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""  #데이터 셋의 문장 시작 3
        examples = [] #예시 리스트 주고
        for (i, line) in enumerate(lines[1:]): #enumerate 줌
            line = line.split("\t")
            guid = "%s-%s" % (set_type, i)
            text_a = line[1]
            label = line[2]
            if i % 10000 == 0:
                logger.info(line)
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples

    def get_examples(self, mode):
        """
        Args:
            mode: train, dev, test
        """ #데이터 셋의 문장 시작 4
        file_to_read = None #파일 읽기 모드로
        if mode == "train":
            file_to_read = self.args.train_file
        elif mode == "dev":
            file_to_read = self.args.dev_file
        elif mode == "test":
            file_to_read = self.args.test_file
        
        logger.info("LOOKING AT {}".format(os.path.join(self.args.data_dir, self.args.task, file_to_read)))
        return self._create_examples(
            self._read_file(os.path.join(self.args.data_dir, self.args.task, file_to_read)), mode
        )


모델 설정

In [ ]:
class InputExample(object):
    """
    A single training/test example for simple sequence classification.
    """
    #인풋 예시 데이터 문장 시작 1
    def __init__(self, guid, text_a, text_b, label):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary.""" #인풋 예시 데이터 문장 시작 2
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string.""" #인풋 예시 데이터 문장 시작 3
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"


class InputFeatures(object):
    """A single set of features of data.""" 
    # 피처 문장 시작 1
    def __init__(self, input_ids, attention_mask, token_type_ids, label):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.label = label

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary.""" # 피처 문장 시작 2
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string.""" # 피처 문장 시작 3
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"


In [ ]:
import copy
from torch.utils.data import TensorDataset

훈련

In [ ]:
def seq_cls_convert_examples_to_features(args, examples, tokenizer, max_length, task, processor):
    label_list = processor.get_labels()
    logger.info("Using label list {} for task {}".format(label_list, task))

    label_map = {label: i for i, label in enumerate(label_list)} #enumerate 추가

    def label_from_example(example):
        #라벨의 예제의 아웃풋 모드 설정 2개
        if OUTPUT_MODE == "classification":
            return label_map[example.label]
        elif OUTPUT_MODE == "regression":
            return float(example.label)
        raise KeyError(OUTPUT_MODE)

    labels = [label_from_example(example) for example in examples]

    batch_encoding = tokenizer.batch_encode_plus(
        [(example.text_a, example.text_b) for example in examples],
        max_length=max_length,
        padding="max_length",
        # token들의 max length보다 크게 MAX_LEN을 설정
        # 설정한 MAX_LEN 만큼 빈 공간을 0이 채움
        add_special_tokens=True,
        truncation=True,
    )

    features = [] #빈리스트
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}
        feature = InputFeatures(**inputs, label=labels[i])
        features.append(feature)

    for i, example in enumerate(examples[:5]): # i 와 예시에 enumerate 주고
        logger.info("*** Example ***")
        logger.info("guid: {}".format(example.guid))
        logger.info("input_ids: {}".format(" ".join([str(x) for x in features[i].input_ids])))
        logger.info("attention_mask: {}".format(" ".join([str(x) for x in features[i].attention_mask])))
        logger.info("token_type_ids: {}".format(" ".join([str(x) for x in features[i].token_type_ids])))
        logger.info("label: {}".format(features[i].label))
        #결괏값 로거정보


    return features

def load_and_cache_examples(args, tokenizer, mode):
    processor = NsmcProcessor(args) #수정 > 프로세서는 NSMC프로세서로 줄거니 수정

        # Load data features from cache or dataset file

    cached_features_file = os.path.join(
            args.data_dir,
                    "cached_{}_{}_{}_{}".format(
                                str(args.task), list(filter(None, args.model_name_or_path.split("/"))).pop(), str(args.max_seq_len), mode
                                        ),
                                            )



    # Load data features from cache or dataset file
    cached_features_file = os.path.join(
        args.data_dir,
        "cached_{}_{}_{}_{}".format(
            str(args.task), list(filter(None, args.model_name_or_path.split("/"))).pop(), str(args.max_seq_len), mode
        ),
    )

    if os.path.exists(cached_features_file):
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        logger.info("Creating features from dataset file at %s", args.data_dir)
        if mode in ("train", "dev", "test"):
            examples = processor.get_examples(mode)
        else:
            raise ValueError("For mode, only train, dev, test is avaiable")

        features = seq_cls_convert_examples_to_features(
            args, examples, tokenizer, max_length=args.max_seq_len, task=args.task, processor=processor
        )
        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save(features, cached_features_file) #파일 저장

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)

    if OUTPUT_MODE == "classification":
        all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
    elif OUTPUT_MODE == "regression":
        all_labels = torch.tensor([f.label for f in features], dtype=torch.float)

    dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)
    
    return dataset

In [ ]:
logger = logging.getLogger(__name__)

학습

In [ ]:
#모델 학습 데이터 값                                

def train(args, model, train_dataset, dev_dataset=None, test_dataset=None):
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else: 
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
    
    # Prepare optimizer and schedule 
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
         'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 
         'weight_decay': 0.0}
    ]
    #옵티마이저, 스케쥴러 설정
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(t_total * args.warmup_proportion), num_training_steps=t_total)

    if os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt")) and os.path.isfile(
            os.path.join(args.model_name_or_path, "scheduler.pt")
    ):
        # Load optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))
    
    # Train
    # - log 로그 정보   
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Total train batch size = %d", args.train_batch_size)
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)
    logger.info("  Logging steps = %d", args.logging_steps)
    logger.info("  Save steps = %d", args.save_steps)

    global_step = 0
    tr_loss = 0.0

    model.zero_grad() #모델 초기화
    #마스터 바 설정 모델 데이터 훈련값에서 가져오기 epochs
    mb = master_bar(range(int(args.num_train_epochs)))

    #epoch 설정 10회로 설정했음. 
    for epoch in mb:
        epoch_iterator = progress_bar(train_dataloader, parent=mb)
        for step, batch in enumerate(epoch_iterator): #enumerate 값 주고
            model.train()
            batch = tuple(t.to(args.device) for t in batch)
            inputs = {
                "input_ids" : batch[0],
                "attention_mask" : batch[1],
                "token_type_ids" : batch[2],
                "labels" : batch[3]
            }
            #아웃풋, 로스 설정
            outputs = model(**inputs)
            loss = outputs[0]

            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps
            #로스 설정
            loss.backward()
            tr_loss += loss.item()

            if (step + 1) % args.gradient_accumulation_steps == 0 or (
                len(train_dataloader) <= args.gradient_accumulation_steps
                and (step + 1) == len(train_dataloader)
            ):
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                
                #값 스킵하여 가져오기
                optimizer.step()
                scheduler.step()
                model.zero_grad()
                global_step +=1


                if args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    if args.evaluate_test_during_training:
                        evaluate(args, model, test_dataset, "test", global_step)
                    else:
                        evaluate(args, model, test_dataset, "dev", global_step)
                
                if args.save_steps > 0 and global_step % args.save_steps == 0:
                    
                    output_dir = os.path.join(args.output_dir, "checkpoint-{}".format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    ) #hasatter 등장 = 변수 확인
                    model_to_save.save_pretrained(output_dir) #모델 저장

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to {}".format(output_dir))
                    
                    #옵티마이저 저장,조인
                    if args.save_optimizer:
                        torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                        torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                        logger.info("Saving optimizer and scheduler states to {}".format(output_dir))

            if args.max_steps > 0 and global_step > args.max_steps:
                break
        #에폭 몇회 완료 출력               break
                #에폭 몇회 완료 출력
                        mb.write("Epoch {} done".format(epoch + 1))

                                if args.max_steps > 0 and global_step > args.max_steps:
                                            break

                                                return global_step, tr_loss / global_step
        mb.write("Epoch {} done".format(epoch + 1))

        if args.max_steps > 0 and global_step > args.max_steps:
            break

    return global_step, tr_loss / global_step

학습 결과 평가분석

In [ ]:
# Eval!
def evaluate(args, model, eval_dataset, mode, global_step=None):
    results = {}
    #eval_dataset 설정
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # Eval!
    #러닝 eval 모델 훈련값, 글로벌스텝 값 출력1
    if global_step != None:
        logger.info("***** Running evaluation on {} dataset ({} step) *****".format(mode, global_step))
        
    else: 
        logger.info("***** Running evaluation on {} dataset *****".format(mode))
    #출력 2
    logger.info("  Num examples = {}".format(len(eval_dataset)))
    logger.info("  Eval Batch size = {}".format(args.eval_batch_size))

    #eval loss 관련
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    
    for batch in progress_bar(eval_dataloader):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
                "labels": batch[3]
            }
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs["labels"].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    if OUTPUT_MODE == "classification":
        preds = np.argmax(preds, axis=1)
    elif OUTPUT_MODE == "regression":
        preds = np.squeeze(preds)
    result = compute_metrics(out_label_ids, preds)
    results.update(result)

    output_dir = os.path.join(args.output_dir, mode)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    output_eval_file = os.path.join(output_dir, "{}-{}.txt".format(mode, global_step) if global_step else "{}.txt".format(mode))
    with open(output_eval_file, "w") as f_w:
        logger.info("***** Eval results on {} dataset *****".format(mode))
        for key in sorted(results.keys()):
            logger.info("  {} = {}".format(key, str(results[key])))
            f_w.write("  {} = {}\n".format(key, str(results[key])))

    return results

실습, 실행

In [ ]:
def main(config_file):
    # Read from config file and make args
    with open(config_file) as f:
        args = AttrDict(json.load(f))
    logger.info("Training/evaluation parameters {}".format(args))

    args.output_dir = os.path.join(args.ckpt_dir, args.output_dir)

    init_logger()

    processor = PROCESSOR(args) #수정
    labels = processor.get_labels()

    if OUTPUT_MODE == "regression":
        config = CONFIG_CLASS.from_pretrained(
            args.model_name_or_path,
            num_labels=TASK_NUM_LABELS
        )
    else:
        config = CONFIG_CLASS.from_pretrained(
            args.model_name_or_path,
            num_labels=TASK_NUM_LABELS,
            id2label={str(i): label for i, label in enumerate(labels)},
            label2id={label: i for i, label in enumerate(labels)},
        )

    tokenizer = TOKENIZER_CLASS.from_pretrained(
        args.model_name_or_path,
        do_lower_case=args.do_lower_case
    )
    model = MODEL_FOR_SEQUENCE_CLASSIFICATION.from_pretrained(
        args.model_name_or_path,
        config=config
    )

    # GPU or CPU
    args.device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
    model.to(args.device)

    # Load dataset
    train_dataset = load_and_cache_examples(args, tokenizer, mode="train") if args.train_file else None
    dev_dataset = load_and_cache_examples(args, tokenizer, mode="dev") if args.dev_file else None
    test_dataset = load_and_cache_examples(args, tokenizer, mode="test") if args.test_file else None

    if dev_dataset == None:
        args.evaluate_test_during_training = True  # If there is no dev dataset, only use testset

    if args.do_train:
        global_step, tr_loss = train(args, model, train_dataset, dev_dataset, test_dataset)
        logger.info(" global_step = {}, average loss = {}".format(global_step, tr_loss))

    results = {}
    if args.do_eval:
        checkpoints = list(
            os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + "pytorch_model.bin", recursive=True))
        )
        if not args.eval_all_checkpoints:
            checkpoints = checkpoints[-1:]
        else:
            logging.getLogger("transformers.configuration_utils").setLevel(logging.WARN)  # Reduce logging
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1]
            model = MODEL_FOR_SEQUENCE_CLASSIFICATION.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, test_dataset, mode="test", global_step=global_step)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

        output_eval_file = os.path.join(args.output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as f_w:
            for key in sorted(results.keys()):
                f_w.write("{} = {}\n".format(key, str(results[key])))

In [ ]:
TASK_NUM_LABELS = 1
PROCESSOR = NsmcProcessor
OUTPUT_MODE = "classification"
CONFIG_CLASS = ElectraConfig
TOKENIZER_CLASS = ElectraTokenizer
MODEL_FOR_SEQUENCE_CLASSIFICATION = ElectraForSequenceClassification

In [ ]:
config_file = '/content/koelectra-small-v3.json'
main(config_file)

04/06/2021 07:31:27 - INFO - __main__ -   Training/evaluation parameters AttrDict({'task': 'nsmc', 'data_dir': '/content/drive/MyDrive/nsmc_0406', 'ckpt_dir': '/content/drive/MyDrive/nsmc_0406/ckpt', 'train_file': 'ratings_train.txt', 'dev_file': '', 'test_file': 'ratings_test.txt', 'evaluate_test_during_training': True, 'eval_all_checkpoints': True, 'save_optimizer': False, 'do_lower_case': False, 'do_train': True, 'do_eval': True, 'max_seq_len': 128, 'num_train_epochs': 10, 'weight_decay': 0.0, 'gradient_accumulation_steps': 1, 'adam_epsilon': 1e-08, 'warmup_proportion': 0, 'max_steps': -1, 'max_grad_norm': 1.0, 'no_cuda': False, 'model_type': 'koelectra-small-v3', 'model_name_or_path': 'monologg/koelectra-small-v3-discriminator', 'output_dir': '/content/drive/MyDrive/nsmc_0406/koelectra-small-v3-nsmc-ckpt', 'seed': 42, 'train_batch_size': 32, 'eval_batch_size': 128, 'logging_steps': 2000, 'save_steps': 2000, 'learning_rate': 5e-05})
04/06/2021 07:31:27 - INFO - filelock -   Lock 140

04/06/2021 07:31:27 - INFO - filelock -   Lock 140088246315152 released on /root/.cache/huggingface/transformers/bd0f09888c5a5619ddb9de81d4a9936a94e5f45064f9a23ba6d39241ceebce02.d2485d28e5c07ca60bfa4fe84af673e0df83401e5c56bcdd991878cb4966eb34.lock
04/06/2021 07:31:27 - INFO - filelock -   Lock 140088051414096 acquired on /root/.cache/huggingface/transformers/32dc9196217c0cc26c7dd705168e8615ea2d82613aa5b672d7647b8e8d58545f.541023ff50f833a9bab3e48e78ae1856cf6744bdb336c86e797eaf675b62b2b8.lock


04/06/2021 07:31:27 - INFO - filelock -   Lock 140088051414096 released on /root/.cache/huggingface/transformers/32dc9196217c0cc26c7dd705168e8615ea2d82613aa5b672d7647b8e8d58545f.541023ff50f833a9bab3e48e78ae1856cf6744bdb336c86e797eaf675b62b2b8.lock
04/06/2021 07:31:27 - INFO - filelock -   Lock 140087881144848 acquired on /root/.cache/huggingface/transformers/a6c32c62ff893fb2aa32dabc5722c9f9eb7243cc1cb4514b9ba3fdb1b52704d6.35f013c4fd3572cfdddbbdf6223ef162dd4fb536bf83007533f201addf3287b7.lock


04/06/2021 07:31:27 - INFO - filelock -   Lock 140087881144848 released on /root/.cache/huggingface/transformers/a6c32c62ff893fb2aa32dabc5722c9f9eb7243cc1cb4514b9ba3fdb1b52704d6.35f013c4fd3572cfdddbbdf6223ef162dd4fb536bf83007533f201addf3287b7.lock


04/06/2021 07:31:27 - INFO - filelock -   Lock 140087881144592 acquired on /root/.cache/huggingface/transformers/052bc3ecf8c8484f1519650794b32b6d2c70750bcba71d1f763951514b5cf0c8.84bb33167d2e89d46f4cde129b2f4c447618ac33ac46f2012ee9e5e706fec112.lock


...

04/06/2021 09:59:22 - INFO - __main__ -   ***** Eval results on test dataset *****
04/06/2021 09:59:22 - INFO - __main__ -     acc = 0.89056
04/06/2021 09:59:23 - INFO - __main__ -   ***** Running evaluation on test dataset (44000 step) *****
04/06/2021 09:59:23 - INFO - __main__ -     Num examples = 50000
04/06/2021 09:59:23 - INFO - __main__ -     Eval Batch size = 128


04/06/2021 10:00:26 - INFO - __main__ -   ***** Eval results on test dataset *****
04/06/2021 10:00:26 - INFO - __main__ -     acc = 0.89038
04/06/2021 10:00:27 - INFO - __main__ -   ***** Running evaluation on test dataset (46000 step) *****
04/06/2021 10:00:27 - INFO - __main__ -     Num examples = 50000
04/06/2021 10:00:27 - INFO - __main__ -     Eval Batch size = 128


04/06/2021 10:01:30 - INFO - __main__ -   ***** Eval results on test dataset *****
04/06/2021 10:01:30 - INFO - __main__ -     acc = 0.89072
04/06/2021 10:01:31 - INFO - __main__ -   ***** Running evaluation on test dataset (6000 step) *****
04/06/2021 10:01:31 - INFO - __main__ -     Num examples = 50000
04/06/2021 10:01:31 - INFO - __main__ -     Eval Batch size = 128


04/06/2021 10:02:34 - INFO - __main__ -   ***** Eval results on test dataset *****
04/06/2021 10:02:34 - INFO - __main__ -     acc = 0.87968
04/06/2021 10:02:34 - INFO - __main__ -   ***** Running evaluation on test dataset (8000 step) *****
04/06/2021 10:02:34 - INFO - __main__ -     Num examples = 50000
04/06/2021 10:02:34 - INFO - __main__ -     Eval Batch size = 128


04/06/2021 10:03:38 - INFO - __main__ -   ***** Eval results on test dataset *****
04/06/2021 10:03:38 - INFO - __main__ -     acc = 0.88698
